# Strands and Agent-to-Agent (A2A) Protocol

Strands Agents supports the [Agent-to-Agent (A2A)](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/agent-to-agent/), an open standard that defines how AI agents can discover, communicate, and collaborate with each other. Enabling seamless communication between AI agents across different platforms and implementations.

### What is Agent-to-Agent (A2A) Communication?

Agent-to-Agent (A2A) communication is a protocol that enables agents to interact with each other, allowing them to:
- Delegate specialized tasks to other agents
- Share information and context
- Collaborate on complex problems
- Create hierarchical agent structures

![a2a](image/a2a.png)

### Use Cases
- A2A protocol support enables several powerful use cases:
- Multi-Agent Workflows: Chain multiple specialized agents together
- Agent Marketplaces: Discover and use agents from different providers
- Cross-Platform Integration: Connect Strands agents with other A2A-compatible systems
- Distributed AI Systems: Build scalable, distributed agent architectures

### Learn more about the A2A protocol:

- [A2A GitHub Organization](https://github.com/a2aproject/A2A)
- [A2A Python SDK](https://github.com/a2aproject/a2a-python)
- [A2A Documentation](https://a2aproject.github.io/A2A/latest/)


## Understanding A2A Architecture

The A2A architecture consists of several key components:

1. **Primary Agent**: The agent that initiates the communication and delegates tasks.
2. **Secondary Agent(s)**: The agent(s) that receive tasks and provide responses.
3. **A2A Tool**: The component that facilitates communication between agents.
4. **Message Protocol**: The format and structure of messages exchanged between agents.

> A2A and the Model Context Protocol (MCP) are complementary standards for building robust agentic applications:

> ⭐ MCP (Model Context Protocol): Connects agents to tools, APIs, and resources with structured inputs/outputs. Think of it as the way agents access their capabilities.

> ⭐ A2A (Agent2Agent Protocol): Facilitates dynamic, multimodal communication between different agents as peers. It's how agents collaborate, delegate, and manage shared tasks. [Source](https://a2a-protocol.org/latest/#why-a2a-matters)

## ✅ Let's start

In this section, we will do a walkthrough of selected code blocks, that have been used to build [strands-a2a-inter-agent](strands-a2a-inter-agent/), based on the [sample-agentic-ai-demos repositorie](https://github.com/aws-samples/sample-agentic-ai-demos/tree/main/modules/strands-a2a-inter-agent) to create a A2A agent.

In our example, we'll explore a three-tier architecture where:
- An [HR Agent](strands-a2a-inter-agent/hr-agent.py) communicates with an Employee Agent
- The [Employee Agent](strands-a2a-inter-agent/employee-agent.py) communicates with an MCP Server
- The [MCP Server](strands-a2a-inter-agent/server.py) provides access to employee data

![a2a_architecture](image/a2a_example.png)

### 1. Employee Data

First, we have a simple module that generates random employee data with skills:

```python
import random

FIRST_NAMES = ["James", "Mary", "John", "Patricia", "Robert", "Jennifer", "Michael", "Linda", "William", "Elizabeth"]
LAST_NAMES = ["Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia", "Miller", "Davis", "Rodriguez", "Martinez"]

SKILLS = {
    "Kotlin", "Java", "Python", "JavaScript", "TypeScript",
    "React", "Angular", "Spring Boot", "AWS", "Docker",
    "Kubernetes", "SQL", "MongoDB", "Git", "CI/CD",
    "Machine Learning", "DevOps", "Node.js", "REST API", "GraphQL"
}

EMPLOYEES = list({emp["name"]: emp for emp in [
    {
        "name": f"{random.choice(FIRST_NAMES)} {random.choice(LAST_NAMES)}",
        "skills": random.sample(list(SKILLS), random.randint(2, 5))
    }
    for i in range(100)
]}.values())

```

### 2. MCP Server

Next, we have an MCP server that exposes tools to access the employee data:


```python
from mcp.server.fastmcp import FastMCP

from employee_data import SKILLS, EMPLOYEES

mcp = FastMCP("employee-server", stateless_http=True, host="0.0.0.0", port=8002)

@mcp.tool()
def get_skills() -> set[str]:
    """all of the skills that employees may have - use this list to figure out related skills"""
    print("get_skills")
    return SKILLS

@mcp.tool()
def get_employees_with_skill(skill: str) -> list[dict]:
    """employees that have a specified skill - output includes fullname (First Last) and their skills"""
    print(f"get_employees_with_skill({skill})")
    skill_lower = skill.lower()
    employees_with_skill = [employee for employee in EMPLOYEES if any(s.lower() == skill_lower for s in employee["skills"])]
    if not employees_with_skill:
        raise ValueError(f"No employees have the {skill} skill")
    return employees_with_skill

if __name__ == "__main__":
    mcp.run(transport="streamable-http")
    
```

The MCP server exposes two tools:
1. `get_skills()`: Returns all possible skills that employees may have
2. `get_employees_with_skill(skill)`: Returns employees that have a specific skill

These tools are exposed via the Model Context Protocol (MCP) using the FastMCP framework, which provides a standardized way for agents to access these functions.

### Server Configuration Options¶
The A2AServer constructor accepts several configuration options:

- **agent**: The Strands Agent to wrap with A2A compatibility
- **host**: Hostname or IP address to bind to (default: "0.0.0.0")
- **port**: Port to bind to (default: 9000)
- **version**: Version of the agent (default: "0.0.1")
- **skills**: Custom list of agent skills (default: auto-generated from tools)

### 3. Employee Agent

The Employee Agent connects to the MCP server and exposes its capabilities through an A2A server:

```python
import os
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands.tools.mcp.mcp_client import MCPClient
from strands.multiagent.a2a import A2AServer
from urllib.parse import urlparse
from strands.models.anthropic import AnthropicModel


# Define URLs correctly
EMPLOYEE_INFO_URL = "http://localhost:8002/mcp/"
EMPLOYEE_AGENT_URL = "http://localhost:8001/"

# Create the MCP client
employee_mcp_client = MCPClient(lambda: streamablehttp_client(EMPLOYEE_INFO_URL))

model = AnthropicModel(
    client_args={
        "api_key": "YOUR_API_KEY_HERE", # Replace with your API key
    },
    max_tokens=1028,
    model_id="claude-sonnet-4-20250514",
    params={
        "temperature": 0.7,
    }
)

# Use the MCP client within a context
with employee_mcp_client:
    tools = employee_mcp_client.list_tools_sync()
    
    # Create a Strands agent
    employee_agent = Agent(
        model=model,
        name="Employee Agent",
        description="Answers questions about employees",
        tools=tools,
        system_prompt="you must abbreviate employee first names and list all their skills"
    )
    
    # Create A2A server
    a2a_server = A2AServer(
        agent=employee_agent, 
        host=urlparse(EMPLOYEE_AGENT_URL).hostname, 
        port=int(urlparse(EMPLOYEE_AGENT_URL).port)
    )
    
    # Start the server
    if __name__ == "__main__":
        a2a_server.serve(host="0.0.0.0", port=8001)

```

Key components of the Employee Agent:

1. **MCP Client**: Connects to the MCP server to access the employee data tools
2. **Strands Agent**: Uses the AnthropicModel to process requests and use the MCP tools
3. **A2A Server**: Exposes the agent's capabilities to other agents through an HTTP API

The Employee Agent acts as a bridge between the HR Agent and the MCP Server, providing a more intelligent interface to the raw data.

### 4. HR Agent

Finally, the HR Agent provides a user-facing API and communicates with the Employee Agent:

```python
import os

import uvicorn
from strands import Agent
from strands.models import BedrockModel
from strands_tools.a2a_client import A2AClientToolProvider
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
from pydantic import BaseModel

EMPLOYEE_AGENT_URL = "http://localhost:8001/"

app = FastAPI(title="HR Agent API")

class QuestionRequest(BaseModel):
    question: str

@app.get("/health")
def health_check():
    return {"status": "healthy"}

bedrock_model = BedrockModel(
    model_id="amazon.nova-micro-v1:0",
    region_name="us-east-1",
    temperature=0.5,
)

@app.post("/inquire")
async def ask_agent(request: QuestionRequest):
    async def generate():
        provider = A2AClientToolProvider(known_agent_urls=[EMPLOYEE_AGENT_URL])

        agent = Agent(model=bedrock_model, tools=provider.tools)

        stream_response = agent.stream_async(request.question)

        async for event in stream_response:
            if "data" in event:
                yield event["data"]

    return StreamingResponse(
        generate(),
        media_type="text/plain"
    )

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

```

## ✅ Running the Example

To run this example, you need to start each component in a separate terminal:

1. Start the MCP Server:
```bash
python server.py
```

2. Start the Employee Agent:
```bash
python employee-agent.py
```

3. Start the HR Agent:
```bash
python hr-agent.py
```

4. Make a request to the HR Agent:
```bash
curl -X POST --location "http://0.0.0.0:8000/inquire" \
-H "Content-Type: application/json" \
-d '{"question": "list employees that have skills related to AI programming"}'
```

## Best Practices for A2A Tools

When using A2A tools with the Strands Agent Framework, consider the following best practices:

1. **Clear Agent Roles**: Define clear roles and responsibilities for each agent in your system.

2. **Effective Delegation**: Ensure that the coordinator agent knows when and how to delegate tasks to specialized agents.

3. **Context Preservation**: Maintain context when passing information between agents to ensure coherent responses.

4. **Error Handling**: Implement robust error handling for agent-to-agent communication.

5. **Avoid Circular Dependencies**: Be careful to avoid circular dependencies between agents that could lead to infinite loops.

6. **Optimize Communication**: Minimize the number of agent-to-agent communications to reduce latency and costs.

7. **Monitor Performance**: Monitor the performance of your agent network to identify bottlenecks and opportunities for improvement.

8. **Test Thoroughly**: Test your agent network with a variety of inputs to ensure it handles edge cases correctly.


## Conclusion

In this notebook, we've explored Agent-to-Agent (A2A) communication in depth and learned how to leverage it effectively with the Strands Agent Framework. We've created specialized agents, implemented various A2A patterns, and built complex workflows that demonstrate the power of agent collaboration.

A2A is a powerful paradigm that enables the creation of sophisticated agent networks capable of solving complex problems through collaboration and specialization.